In [1]:
import datetime


import numpy as np
import pandas as pd
import torch
import torch
import torchmetrics
import transformers
import torch.nn as nn
from tqdm.auto import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

import utils

C:\Users\royta\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
model.to(device)

def tokenize_function(examples, max_length):
    return tokenizer(examples, max_length=max_length, padding="max_length", truncation=True, return_tensors='pt')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [4]:
import torchsummary

torchsummary.summary(model);

Layer (type:depth-idx)                   Param #
├─BertModel: 1-1                         --
|    └─BertEmbeddings: 2-1               --
|    |    └─Embedding: 3-1               (22,268,928)
|    |    └─Embedding: 3-2               (393,216)
|    |    └─Embedding: 3-3               (1,536)
|    |    └─LayerNorm: 3-4               (1,536)
|    |    └─Dropout: 3-5                 --
|    └─BertEncoder: 2-2                  --
|    |    └─ModuleList: 3-6              (85,054,464)
|    └─BertPooler: 2-3                   --
|    |    └─Linear: 3-7                  (590,592)
|    |    └─Tanh: 3-8                    --
├─Dropout: 1-2                           --
├─Linear: 1-3                            1,538
Total params: 108,311,810
Trainable params: 1,538
Non-trainable params: 108,310,272


In [5]:
df = pd.read_csv('data/train.csv')
df['len'] = df['text'].apply(lambda x: len(x))
df[df['location'].notnull()]

,id,keyword,location,text,target,len
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,55
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,67
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,82
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,34
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,76
...,...,...,...,...,...,...
7575,10826,wrecked,TN,On the bright side I wrecked http://t.co/uEa0t...,0,51
7577,10829,wrecked,#NewcastleuponTyne #UK,@widda16 ... He's gone. You can relax. I thoug...,0,107
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0,107
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0,93


# Classifier on BERT's embeddings

### work with the data

In [42]:
config = {
    'd': [64, 32],
    'dropout': 0.5,
    'n_components': 384,
    'batch_size': 1024,
}

In [7]:
x = tokenize_function(df['text'].tolist(), df['len'].max())

In [8]:
X = []
batch_size = 512

num_batches = len(df)//batch_size
if len(df)%batch_size != 0:
    num_batches+= 1
    
model.eval()
with torch.no_grad():
    for batch in range(num_batches):
        inputs = {
            k: v[batch*batch_size:(batch+1)*batch_size].to(device) 
                  for k, v in x.items()
        }
        
        outputs = model.bert(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            token_type_ids=inputs['token_type_ids'],
        )
        
        X.append(outputs.pooler_output.cpu())

In [9]:
torch.cuda.empty_cache()
X = torch.cat(X)
Y = df['target'].values

In [10]:
Xtrain, Xval, Ytrain, Yval = train_test_split(X, Y, test_size=0.1, random_state=69)

In [11]:
train_dataset = utils.EmbeddingDataset(Xtrain, Ytrain)
val_dataset = utils.EmbeddingDataset(Xval, Yval)

In [12]:
def validate(name):    
    print(name)

    checkpoint = torch.load(f'Models/W/{name}.torch')
    classification_model.load_state_dict(checkpoint['model'])

    val_pred, val_true = trainer.val(val_dataloader)
    val_pred = val_pred.softmax(1)
    print('AUC:', auc(val_pred, val_true))
    print('ACC:', acc(val_pred, val_true))

In [43]:
n_components = config['n_components']
if n_components is not None:
    pca = PCA(n_components=n_components)
    pca.fit(Xtrain)
    
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=config['batch_size'])
val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'])

classification_model = utils.ClassifierModel(
    input_dim=n_components if n_components is not None else train_dataset.shape()[1], 
    callable_=None if n_components is None else pca.transform,
    output_dim=2,
    **config,
).to(device)

trainer = utils.Trainer(
    model=classification_model,
    metric=torchmetrics.AUROC(num_classes=2),
    loss_fn=nn.CrossEntropyLoss(reduce=True),
    optimizer=torch.optim.AdamW(classification_model.parameters(), lr=3e-4),
)

acc = torchmetrics.Accuracy(num_classes=2)
auc = torchmetrics.AUROC(num_classes=2)

C:\Users\royta\AppData\Local\Programs\Python\Python310\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\royta\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [44]:
name = 'BERT embeddings + {0} + {1} + {2} + {3} '.format(
    f"{config['n_components']} pca", 
    f"{config['d']} d",
    f"{config['dropout']} dropout",
    f"{config['batch_size']} batchsize",
)
board_name = name + datetime.datetime.now().strftime("%Y.%m.%d - %H-%M-%S")

log_dir = f"logs/fit/{board_name}"
writer = SummaryWriter(log_dir)

In [45]:
try:
    wait = 0
    patience = 150

    epoch = 0
    best_loss = -np.inf
    while wait < patience:
        train_loss = trainer.train(train_dataloader, epoch)

        val_pred, val_true = trainer.val(val_dataloader)
        val_pred = val_pred.softmax(1)
        metrics = {
            'AUC': auc(val_pred, val_true),
            'ACC': acc(val_pred, val_true),
        }
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('AUC/train', trainer.metric.compute(), epoch)
        writer.add_scalar('AUC/val', metrics['AUC'], epoch)
        writer.add_scalar('ACC/val', metrics['ACC'], epoch)


        wait+=1
        epoch+=1
        if metrics['AUC'] > best_loss:
            checkpoint = trainer.checkpoint()
            torch.save(checkpoint, f'Models/W/{name}.torch')
            best_loss = metrics['AUC']
            wait = 0

except KeyboardInterrupt:
    print("KeyboardInterrupt")

In [46]:
validate(name)

BERT embeddings + 384 pca + [64, 32] d + 0.5 dropout + 1024 batchsize 
AUC: tensor(0.8214)
ACC: tensor(0.7717)


In [38]:
validate(name)

BERT embeddings + 512 pca + [64, 32] d + 0.15 dropout + 1024 batchsize 
AUC: tensor(0.8257)
ACC: tensor(0.7730)


In [33]:
# 2048 batch
validate(name)

BERT embeddings + 256 pca + [] d + 0.15 dropout 
AUC: tensor(0.8196)
ACC: tensor(0.7743)


In [28]:
# 2048 batch
validate(name)

BERT embeddings + 256 pca + [64, 32] d + 0.15 dropout 
AUC: tensor(0.8231)
ACC: tensor(0.7808)


In [23]:
# 2048 batch
validate(name)

BERT embeddings + 256 pca + [128, 128, 64] d + 0.15 dropout 
AUC: tensor(0.8211)
ACC: tensor(0.7835)


In [18]:
# 2048 batch
validate(name)

BERT embeddings + 256 pca + [128, 64] d + 0.15 dropout 
AUC: tensor(0.8247)
ACC: tensor(0.7782)


In [42]:
# 2048 batch
# 0.5 dropout
validate(name)

BERT embeddings + 256 pca + [128, 64] d 
AUC: tensor(0.8223)
ACC: tensor(0.7769)


In [37]:
# 2048 batch
# 0.5 dropout
validate(name)

BERT embeddings + 512 pca + [128, 64] d 
AUC: tensor(0.8231)
ACC: tensor(0.7769)


In [32]:
# 2048 batch
# 0.5 dropout
validate(name)

BERT embeddings + None pca + [128, 64] d
AUC: tensor(0.8180)
ACC: tensor(0.7677)


# Tune BERT

w.i.p.

In [51]:
class NLPDataset(Dataset):
    def __init__(self, df, tokenizer, batch_size, **kwargs):
        self.df = df.reset_index()
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.values = df[['text', 'target']].values
        
        
    def tokenize_function(self, examples):
        return tokenizer(examples, padding="max_length", truncation=True, return_tensors='pt')

    
    def __len__(self):
        return len(self.df)
    
    
    def __getitem__(self, idx):
        idx = random.choices(range(len(self)), k=self.batch_size)
        x = self.tokenize_function(self.values[idx, 0].tolist())
        y = self.values[idx, 1].astype('int32')
        
        x['labels'] = torch.LongTensor(y)
        return x

In [9]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = transformers.get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [11]:
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(
            labels=batch['labels'],
            input_ids=batch['input_ids'][0],
            attention_mask=batch['attention_mask'][0],
            token_type_ids=batch['token_type_ids'][0],
        )
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  1%|▍                                                                          | 120/19839 [06:35<18:25:30,  3.36s/it]

KeyboardInterrupt: 

# End to end transformer classifier

w.i.p.